# 1. 데이터 로드

In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format) # 항상 float 형식으로
pd.set_option("display.max_columns",100)

import numpy as np 

# df plot
import cufflinks as cf 
import plotly.plotly as py 
import matplotlib.pyplot as plt 
import seaborn as sns 

# warnings 
import warnings ; warnings.filterwarnings("ignore")

# style
plt.style.use("ggplot")
cf.go_offline()

In [2]:
# 한글 폰트 출력 
import matplotlib
from matplotlib import font_manager, rc 
import platform 

font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc("font",family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

In [3]:
# 데이터 로드 
path = "C:/Users/sk_jo/Desktop/Project/titanic/"

def load_dataset(): 
    sales = pd.read_excel(path+"sales.xlsx",header =1)
    view_rate = pd.read_excel(path+"view_rate.xlsx",header =1, index_col= 0)
    return sales, view_rate

sales = load_dataset()[0]
view_rate = load_dataset()[1]

# 2. 데이터 전처리 

In [4]:
import datetime 
## TODO 
## 1.무형 상품은 추정 제외 
## 2.결측된 노출분 채우기 (직전의 값 집어넣기) 
## 3. 판매단가 > 취급액이면 취급액 1 
## 4. 취급액이 50000인 경우와 1로 처리
## 해당 경우 판매가 이뤄지지 않아서 해당 값을 부여했다고 했기 떄문에 안팔리는 것은 안팔릴 것이라고 예측을 해줘야 한다.
##  0 일경우 metric이 mape일 때 nan값이 발생하기 떄문에 1로 scaling 
## 5. 시간 데이터 처리
### 



def filling_missing_exposure(df):
    #2번 전처리
    for i,row in df.iterrows():
        val = row["노출(분)"]
        if i == 0:
            continue 
        if np.isnan(val):
            if df.loc[i,"방송일시"] == df.loc[i-1,"방송일시"]:
                df.loc[i,"노출(분)"] = df.loc[i-1,"노출(분)"]
            else :
                continue 
    return df 

def custom(price, total):
    # 3전처리 
    if price > total: return 1
    else: return total
    
def handle_date(df):
    # 5번 전처리
    df["방송일시"] = pd.to_datetime(df["방송일시"])
    df['month'] = pd.DatetimeIndex(df['방송일시']).month
    df['day'] = pd.DatetimeIndex(df['방송일시']).day
    df['hour'] = pd.DatetimeIndex(df['방송일시']).hour
    df['minute'] = pd.DatetimeIndex(df['방송일시']).minute
    df['weekday'] = pd.DatetimeIndex(df['방송일시']).weekday
    
    return df 

def deleting_data(df):
    df = df.replace({"취급액":50000},{"취급액":1})
    df.drop(["마더코드","상품코드"],axis=1,inplace=True)
    return df 
    

def handle_holiday(df):
    # 4번 전처리 
    hol = ['2019-01-01','2019-02-04','2019-02-05','2019-02-06','2019-03-01','2019-05-05','2019-05-06'
           ,'2019-05-12','2019-06-06','2019-08-15','2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09'
           ,'2019-12-25','2020-01-01']
    df["holiday"] = df["방송일시"].apply(lambda x : 1 if ((datetime.datetime.strftime(x,"%Y-%m-%d")) in hol) else 0 )
    
    return df

    
def pre_processing(df): 
    # 1. 무형 상품 추정 제외 
    df = df.drop(df[df["상품군"]=="무형"].index,axis = 0)
    # 2. 결측된 노출분 채우기 
    df = filling_missing_exposure(df)
                
    # 3. 판매단가 > 취급액이면 취급액 1 
    df["취급액"] = df.apply(lambda x : custom(x["판매단가"], x["취급액"]),axis=1)
    # 4. month, day, hour, minute, weekday 열 추가 
    df = handle_date(df)
    # 5. 공휴일 처리
    df = handle_holiday(df)
    
    # 6. data 삭제 
    df = deleting_data(df)
    
    
    # reset_index
    df = df.reset_index(drop=True)
    
    return df 

sales = pre_processing(sales)
sales.head()

,방송일시,노출(분),상품명,상품군,판매단가,취급액,month,day,hour,minute,weekday,holiday
0,2019-01-01 06:00:00,20.00,테이트 남성 셀린니트3종,의류,39900,2099000.00,1,1,6,0,1,1
1,2019-01-01 06:00:00,20.00,테이트 여성 셀린니트3종,의류,39900,4371000.00,1,1,6,0,1,1
2,2019-01-01 06:20:00,20.00,테이트 남성 셀린니트3종,의류,39900,3262000.00,1,1,6,20,1,1
3,2019-01-01 06:20:00,20.00,테이트 여성 셀린니트3종,의류,39900,6955000.00,1,1,6,20,1,1
4,2019-01-01 06:40:00,20.00,테이트 남성 셀린니트3종,의류,39900,6672000.00,1,1,6,40,1,1


# 3. 외부 데이터 가지고 오기

## 3-1 날씨 데이터

출처 : 케이웨더 기상청 자료

In [5]:
weather = pd.read_csv("./data_train/weathers_train.csv")
weather.head()

,방송일시,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지 주의보 및 경보,평균기온(℃),최고기온(℃),최저기온(℃),season,강수량(mm),폭염주의보,한파주의보,대설주의보,호우주의보
0,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0
1,2019-01-02,33.08,21.36,0.01,0.04,0.68,0.00,0,-4.90,0.20,-8.80,겨울,0.00,0,1,0,0
2,2019-01-03,38.80,23.36,0.01,0.04,0.74,0.01,0,-3.50,3.20,-8.40,겨울,0.00,0,1,0,0
3,2019-01-04,59.44,40.36,0.00,0.06,1.06,0.01,0,-1.10,4.10,-6.20,겨울,0.00,0,1,0,0
4,2019-01-05,65.72,42.52,0.02,0.03,0.64,0.01,1,-2.80,1.10,-5.50,겨울,0.00,0,1,0,0


In [6]:
weather.rename(columns = {"방송일시":"방송날짜"},inplace=True)

In [7]:
sales["방송날짜"] = sales["방송일시"].apply(lambda x: datetime.datetime.strftime(x,"%Y-%m-%d"))
sales = sales. merge(weather,on ="방송날짜")
sales.head()

,방송일시,노출(분),상품명,상품군,판매단가,취급액,month,day,hour,minute,weekday,holiday,방송날짜,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지 주의보 및 경보,평균기온(℃),최고기온(℃),최저기온(℃),season,강수량(mm),폭염주의보,한파주의보,대설주의보,호우주의보
0,2019-01-01 06:00:00,20.00,테이트 남성 셀린니트3종,의류,39900,2099000.00,1,1,6,0,1,1,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0
1,2019-01-01 06:00:00,20.00,테이트 여성 셀린니트3종,의류,39900,4371000.00,1,1,6,0,1,1,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0
2,2019-01-01 06:20:00,20.00,테이트 남성 셀린니트3종,의류,39900,3262000.00,1,1,6,20,1,1,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0
3,2019-01-01 06:20:00,20.00,테이트 여성 셀린니트3종,의류,39900,6955000.00,1,1,6,20,1,1,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0
4,2019-01-01 06:40:00,20.00,테이트 남성 셀린니트3종,의류,39900,6672000.00,1,1,6,40,1,1,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0


In [8]:
sales.drop(["방송날짜"],axis=1,inplace=True)

## 3-2 카테고리 데이터 

출처 : 쿠팡 검색엔진 서비스

In [9]:
category = pd.read_excel("./data_train/category_train.xlsx",index_col=[0]).reset_index(drop=True)
category.head()

,상품명,상품군,cat_1,cat_2,cat_3
0,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,가구,결혼준비,가구/침구,침실가구
1,보루네오 루나 유로탑 멀티수납형 LED 침대 Q 퀸,가구,결혼준비,가구/침구,침실가구
2,보루네오 루나 유로탑 멀티수납형 LED 침대 K 킹,가구,결혼준비,가구/침구,침실가구
3,(일) 삼익가구 LED 제니비 서랍형 침대 SS,가구,결혼준비,가구/침구,침실가구
4,(무) 삼익가구 LED 제니비 서랍형 침대 SS,가구,결혼준비,가구/침구,침실가구


In [10]:
def category_extract(df,product_name):
    try:
        cat = df[df["상품명"]==product_name]["cat_3"].values[0]
    except:
        cat = ''
    return cat

sales["상품군-중"] = sales.apply(lambda x : category_extract(category,x["상품명"]),axis=1)
sales.head()

,방송일시,노출(분),상품명,상품군,판매단가,취급액,month,day,hour,minute,weekday,holiday,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지 주의보 및 경보,평균기온(℃),최고기온(℃),최저기온(℃),season,강수량(mm),폭염주의보,한파주의보,대설주의보,호우주의보,상품군-중
0,2019-01-01 06:00:00,20.00,테이트 남성 셀린니트3종,의류,39900,2099000.00,1,1,6,0,1,1,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0,니트
1,2019-01-01 06:00:00,20.00,테이트 여성 셀린니트3종,의류,39900,4371000.00,1,1,6,0,1,1,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0,니트
2,2019-01-01 06:20:00,20.00,테이트 남성 셀린니트3종,의류,39900,3262000.00,1,1,6,20,1,1,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0,니트
3,2019-01-01 06:20:00,20.00,테이트 여성 셀린니트3종,의류,39900,6955000.00,1,1,6,20,1,1,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0,니트
4,2019-01-01 06:40:00,20.00,테이트 남성 셀린니트3종,의류,39900,6672000.00,1,1,6,40,1,1,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울,0.00,0,1,0,0,니트


## 3-3 시청률 데이터

출처 : 구글 검색 데이터 + 닐슨 데이터 

In [11]:
view_rate = pd.read_excel("./data_train/view_rate_train.xlsx")
view_rate.head()

,Unnamed: 0,view_rate,date,start time,finish time,rank
0,비켜라 운명아,14.40,20190101,20:30,21:00,1
1,특선영화,12.50,20190101,19:30,22:00,2
2,특선영화,11.20,20190101,19:30,22:00,3
3,비켜라 운명아,17.80,20190102,20:30,21:00,1
4,황후의 품격,14.35,20190102,22:00,23:10,2


## 3-4 경제지표 데이터

출처 : 한국은행 경제통계시스템

### 개인 신용카드 사용액

In [12]:
economic_indicators = {'month':[1,2,3,4,5,6,7,8,9,10,11,12],
    'ec_credit' :[52039328,45466358,51678679,50723386,53425506,50508891,53730267,52485647,50893769,54017093,53814165, 55909956]}

### 소매 판매액 지수(경기 동행지표) 

In [13]:
economic_indicators["retail_index"] = [109.8,99.8,116.0,111.4,116.6,110.0,110.5,111.4,113.3,115.8,120.5,121.6]

### ns 홈쇼핑 분기 매출액(단위 백만원) 

In [14]:
economic_indicators["sales_by_quarter"] = [117699,117699,117699,124570,124570,124570,124049,124049,124049,131669,131669,131669]

### 경기동행지수순환변동치 

In [15]:
economic_indicators["coincident_index"] = [100.2,99.8,99.7,99.7,99.9,99.9,99.8,99.9,100.0,100.0,99.9,100.2]

In [16]:
#데이터 합치기
ec_df = pd.DataFrame(economic_indicators)
sales = sales.merge(ec_df,on="month")

In [68]:
# sales.to_excel("C:/Users/sk_jo/Desktop/sales_중간.xlsx",encoding= "cp949")

# 4. Feature engineering 

## 4-1. 상품군 카테고리 데이터